<a href="https://colab.research.google.com/github/wusinyee/MW-Portfolio/blob/main/TCCADBv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash plotly pandas numpy pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 24.1 MB/s eta 0:00:00


In [ ]:
!pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.2 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from google.colab import output
import IPython

# Load your data
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Convert 'TotalCharges' to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# If there are any NaN values after conversion, you might want to handle them
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

print(df['TotalCharges'].dtype)

# Define plotting functions
# (Keep all your plotting functions as they are)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Telco Customer Churn Analysis Dashboard"),

    html.H2("Overall Churn Distribution"),
    dcc.Graph(id='churn-distribution'),

    html.H2("Categorical Variable Analysis"),
    dcc.Dropdown(
        id='categorical-feature',
        options=[{'label': col, 'value': col} for col in df.select_dtypes(include=['object']).columns if col != 'customerID'],
        value='InternetService',
        style={'width': '50%'}
    ),
    dcc.Graph(id='categorical-plot'),
    dcc.Graph(id='churn-rate-plot'),

    html.H2("Numerical Variable Analysis"),
    dcc.Dropdown(
        id='numerical-feature',
        options=[{'label': col, 'value': col} for col in df.select_dtypes(include=[np.number]).columns],
        value='tenure',
        style={'width': '50%'}
    ),
    dcc.Graph(id='numerical-plot'),

    html.H2("Correlation Heatmap"),
    dcc.Graph(id='correlation-heatmap'),

    html.H2("Tenure vs Monthly Charges"),
    dcc.Graph(id='tenure-charges-plot')
])

# Callbacks
@app.callback(
    Output('churn-distribution', 'figure'),
    Input('categorical-feature', 'value')
)
def update_churn_distribution(dummy):
    return plot_churn_distribution(df)

@app.callback(
    [Output('categorical-plot', 'figure'),
     Output('churn-rate-plot', 'figure')],
    Input('categorical-feature', 'value')
)
def update_categorical_plots(feature):
    return plot_categorical_distribution(df, feature), plot_churn_rate_by_category(df, feature)

@app.callback(
    Output('numerical-plot', 'figure'),
    Input('numerical-feature', 'value')
)
def update_numerical_plot(feature):
    return plot_numerical_distribution(df, feature)

@app.callback(
    Output('correlation-heatmap', 'figure'),
    Input('categorical-feature', 'value')
)
def update_correlation_heatmap(dummy):
    return plot_correlation_heatmap(df)

@app.callback(
    Output('tenure-charges-plot', 'figure'),
    Input('categorical-feature', 'value')
)
def update_tenure_charges_plot(dummy):
    return plot_tenure_vs_charges(df)

# Run the app
if __name__ == '__main__':
    port = 8050
    output.serve_kernel_port_as_window(port)
    app.run_server(port=port, debug=True, use_reloader=False)
    display(IPython.display.HTML(f'<a href="https://localhost:{8050}" target="_blank">Click here to open the dashboard</a>'))

float64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>